In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
import unicodedata
from ast import literal_eval
import numpy as np

In [55]:
df_1 = pd.read_csv("Dataset_Proyecto1\DATA_EDA.csv",  index_col=0)

In [56]:
df1 = df_1[["title", "genres", "overview","popularity" ]]


Configuración de funciones para datos de entrenamiento de MLConseguir una sopa de letras

In [ ]:
df1.title = df1.title.str.lower()# convierte todos los títulos a minúsculas
df1.genres = df1.genres.str.replace(" ","")# elimina los espacios en blanco
def safe_literal_eval(value):#función verifica si el valor es una cadena no nula 
    if isinstance(value, str) and not pd.isna(value):#garantiza que no se produzca un error al evaluar los valores nan 
        return literal_eval(value)
    return []
df1.genres = df1.genres.apply(safe_literal_eval)  # Establecer el tipo de datos correcto en la columna genres
df1["features"] = df1.genres  # crea una nueva columna


Conseguir una sopa de letras

In [ ]:
df1['genres'] = df1['genres'].apply(lambda x: ' '.join(x))  # Unir elementos de "genres"
df1['features'] = df1['genres'] + ' ' + df1['overview']  # Unir elementos de "genres" y "overview" en la columna "features"(caracteristicas)

In [60]:
df1 = df1.sort_values("popularity", ascending=False)  # Ordenar por popularidad en orden descendente
df1.drop(["genres", "overview", "popularity"], inplace=True, axis=1)  # Eliminar columnas que no se utilizarán en el modelo de ML
df1["features"] = df1["features"].str.lower()  # Normalizar los valores en la columna "features" a minúsculas
df1.dropna(subset=["features"], inplace=True) #Elimino filas vacias
df1.reset_index(drop=True, inplace=True)  # Restablecer el índice del DataFrame

In [61]:
df1.head(10)

,title,features
0,minions,f a m i l y a n i m a t i o n a d v e n t ...
1,wonder woman,a c t i o n a d v e n t u r e f a n t a s ...
2,beauty and the beast,f a m i l y f a n t a s y r o m a n c e a ...
3,baby driver,a c t i o n c r i m e after being coerced in...
4,big hero 6,a d v e n t u r e f a m i l y a n i m a t ...
5,deadpool,a c t i o n a d v e n t u r e c o m e d y ...
6,guardians of the galaxy vol. 2,a c t i o n a d v e n t u r e c o m e d y ...
7,avatar,a c t i o n a d v e n t u r e f a n t a s ...
8,john wick,a c t i o n t h r i l l e r ex-lunatic john ...
9,gone girl,m y s t e r y t h r i l l e r d r a m a wi...


Exportar conjunto de datos para datos de entrenamiento de ML

In [62]:
df1.to_csv("API1_fucnion7.csv")#exportar dataset

In [27]:
df = pd.read_csv("API1_fucnion7.csv", index_col=0)

In [6]:
df

,title,features
0,minions,f a m i l y a n i m a t i o n a d v e n t ...
1,wonder woman,a c t i o n a d v e n t u r e f a n t a s ...
2,beauty and the beast,f a m i l y f a n t a s y r o m a n c e a ...
3,baby driver,a c t i o n c r i m e after being coerced in...
4,big hero 6,a d v e n t u r e f a m i l y a n i m a t ...
...,...,...
44399,hellyys,c o m e d y finnish soft erotic movie
44400,altar of fire,d o c u m e n t a r y this film records a 12 d...
44401,vehshi jatt,"classic indian film starring aasia, afzaal ah..."
44402,raakh,after witnessing the public humiliation &amp;...


In [65]:
def recomendacion(titulo: str):
    '''Ingresas un nombre de película y te recomienda las similares en una lista'''
    ml = df.head(10000)# Utilizar una muestra debido al costo computacional excesivo si se utiliza todo el conjunto de datos
    indices = ml[["title"]]
    tfidf = TfidfVectorizer(stop_words="english", max_features=10000)
    tfidf_matrix = tfidf.fit_transform(ml["features"])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    titulo = titulo.lower().strip()
    titulo = unicodedata.normalize('NFKD', titulo).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    idx = indices[indices["title"] == titulo]

    if idx.empty:
        raise HTTPException(status_code=404, detail="No se encontró la película en el dataset")

    idy = idx.index[0]
    sim_scores = list(enumerate(cosine_sim[idy])) # Obtener directamente los puntajes de similitud sin enumerar
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movies_indices = [i[0] for i in sim_scores]
    recommendations = list(ml['title'].iloc[movies_indices].str.title())

    return {'lista_recomendada': recommendations}


In [58]:
recomendacion("Batman Vs Dracula")

{'lista_recomendada': ['Batman Returns',
  'Happy Feet Two',
  'The Wrong Trousers',
  'Batman',
  'Batman Unlimited: Animal Instincts']}

In [59]:
recomendacion("toy story")

{'lista_recomendada': ['Toy Story 3',
  'Toy Story 2',
  'The 40 Year Old Virgin',
  'Small Fry',
  'Man On The Moon']}

In [68]:
def recomendacion(titulo: str):
    '''Ingresas un nombre de película y te recomienda las similares en una lista'''
    ml = df.head(10000)  # Utilizar una muestra debido al costo computacional excesivo si se utiliza todo el conjunto de datos
    
    # Obtener el dataset de índices y títulos de las películas
    indices = ml[["title"]]
    
    # Configurar el vectorizador tf-idf con palabras comunes en inglés y limitar el número de características
    tfidf = TfidfVectorizer(stop_words="english", max_features=10000)
    
    # Crear la matriz tf-idf con los datos de las características de las películas
    tfidf_matrix = tfidf.fit_transform(ml["features"])
    
    # Calcular la matriz de similitud coseno utilizando el kernel lineal
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Preprocesar el título de la película de entrada
    titulo = titulo.lower().strip()
    titulo = unicodedata.normalize('NFKD', titulo).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Buscar el índice de la película en el dataset
    idx = indices[indices["title"] == titulo]
    
    # Manejar el caso de película no encontrada
    if idx.empty:
        raise HTTPException(status_code=404, detail="No se encontró la película en el dataset")
    
    # Obtener el índice de la película encontrada
    idy = idx.index[0]
    
    # Obtener los puntajes de similitud de la película encontrada con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idy]))
    
    # Ordenar los puntajes de similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de las películas más similares (excluyendo la película de entrada)
    sim_scores = sim_scores[1:6]
    movies_indices = [i[0] for i in sim_scores]
    
    # Obtener los títulos de las películas más similares en forma de lista
    recommendations = list(ml['title'].iloc[movies_indices].str.title())

    return {'lista_recomendada': recommendations}


In [69]:
recomendacion("toy story")

{'lista_recomendada': ['Toy Story 3',
  'Toy Story 2',
  'The 40 Year Old Virgin',
  'Small Fry',
  'Man On The Moon']}